<a href="https://colab.research.google.com/github/leaguilar/AIknowthatfeel/blob/master/Workshop1_example_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Collab

#### git clone

In [1]:
#!git clone https://github.com/<your name>/<your project>.git
#!git clone https://github.com/leaguilar/AIknowthatfeel/ colab
#pip install -r requirements.txt
#import sys
#sys.path.append('colab')

### Import libraries



In [2]:
import numpy as np
import os
import shutil
import datetime

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing.text import one_hot
import pandas as pd
import pickle

In [4]:
from libs.workshop_libs.tensorboard import TrainValTensorBoard

## Import dataset

In [5]:
#Ugly Example preprocessing
pad_char=0
start_char=1
oov_char=2
index_from=3

with open('models/wordDictionary.pkl', 'rb') as handle:
    wordDict = pickle.load(handle)

def encodeList(text_list,vocab_size):
    encoded=[start_char]
    for w in text_list:
        val=wordDict.get(w,oov_char-index_from)+index_from
        if val>=vocab_size:
            val=vocab_size-1
        encoded.append(val)
    #print(encoded)
    return encoded
       
def encodeData(data,vocab_size = 5000):
    data_out=[]
    for index, row in data.iterrows():
        text=(row.iloc[0]+' ### '+row.iloc[1]+' ### '+row.iloc[2])
        text_list=text.split()
        encoded=encodeList(text_list,vocab_size)
        data_out.append(encoded)
    return np.array(data_out)
def encodedLabels(data):
    data_out=[]
    for index, row in data.iterrows():
        if row.iloc[0] == 'others':
            data_out.append([1,0,0,0])
        elif row.iloc[0] == 'happy':
            data_out.append([0,1,0,0])
        elif row.iloc[0] == 'angry':
            data_out.append([0,0,1,0])
        elif row.iloc[0] == 'sad':
            data_out.append([0,0,0,1])
        else:
            raise Exception()
    return np.array(data_out).astype(int)
def prepare_data(x_data,max_conv_length):
    x_data = sequence.pad_sequences(x_data, maxlen=max_conv_length)
    return x_data
def loadAndPrepareData(fname,vocab_size=5000,max_conv_length=50):
    data = pd.read_csv(fname,sep='	',header=None)
    x_data_raw=data.filter([1,2,3], axis=1).copy(deep=True)
    y_data_raw=data.filter([4], axis=1).copy(deep=True)  
    x_data = encodeData(x_data_raw,vocab_size)
    y_data = encodedLabels(y_data_raw)
    x_data = prepare_data(x_data,max_conv_length)                       
    return x_data,y_data

In [6]:
vocab_size=5000
max_conv_length=50

In [7]:
x_train,y_train=loadAndPrepareData('data_train/train.txt',vocab_size,max_conv_length)
x_dev,y_dev=loadAndPrepareData('data_dev/dev.txt',vocab_size,max_conv_length)

In [8]:
len(x_train[0])

50

In [9]:
x_train[0][:40]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   1,  91,  18, 625,   5,  33, 186,   3, 781,  62,  16,
         5], dtype=int32)

In [10]:
len(x_train)

30160

In [11]:
#remove
#max_data=10000
max_data=len(x_train)
x_train=x_train[0:max_data]
y_train=y_train[0:max_data]
x_dev=x_dev[0:max_data]
y_dev=y_dev[0:max_data]

### Check the dataset

In [12]:
## Load tensorboard

In [13]:
%load_ext tensorboard
#%reload_ext tensorboard

The tensorboard module is not an IPython extension.


In [14]:
#!kill 4430

In [15]:
#!ps aux | grep -i tb
logs_base_dir = "./logs"
os.makedirs(logs_base_dir, exist_ok=True)

In [16]:
print(shutil.rmtree(logs_base_dir))

None


In [17]:
os.makedirs(logs_base_dir, exist_ok=True)
%tensorboard --logdir {logs_base_dir}

UsageError: Line magic function `%tensorboard` not found.


In [18]:
### Clean dato, truncate and padding

In [19]:
def create_model(top_words,max_review_length):
    embedding_vector_length = 32
    model = Sequential()
    model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length))
    model.add(LSTM(100))
    model.add(Dense(4, activation='sigmoid'))
    print(model.summary())
    return model

In [20]:
def train_model(model,x_train,y_train,x_dev,y_dev):
        logdir = os.path.join(logs_base_dir, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
        model.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy']
                     )
        model.fit(x_train,
                  y_train,
                  batch_size=64, #64,
                  validation_data=(x_dev, y_dev),
                  nb_epoch=10,
                  callbacks=[TrainValTensorBoard(logdir, 
                                                 histogram_freq=1,
                                                 write_graph=True
                                                )]
                 )

In [21]:
print(x_train.shape)
print(x_dev.shape)

(30160, 50)
(2755, 50)


In [22]:
model=create_model(vocab_size,max_conv_length)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 32)            160000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               53200     
_________________________________________________________________
dense (Dense)                (None, 4)                 404       
Total params: 213,604
Trainable params: 213,604
Non-trainable params: 0
_________________________________________________________________
None


In [23]:
train_model(model,x_train,y_train,x_dev,y_dev)

/home/leo/.local/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 30160 samples, validate on 2755 samples
Epoch 1/10
30160/30160 [==============================] - 19s 643us/step - loss: 0.7282 - acc: 0.7200 - val_loss: 0.3906 - val_acc: 0.8552
Epoch 2/10
30160/30160 [==============================] - 20s 665us/step - loss: 0.3534 - acc: 0.8774 - val_loss: 0.3483 - val_acc: 0.8784
Epoch 3/10
30160/30160 [==============================] - 20s 679us/step - loss: 0.2955 - acc: 0.8982 - val_loss: 0.3651 - val_acc: 0.8733
Epoch 4/10
30160/30160 [==============================] - 21s 684us/step - loss: 0.2631 - acc: 0.9097 - val_loss: 0.4066 - val_acc: 0.8472
Epoch 5/10
30160/30160 [==============================] - 22s 732us/step - loss: 0.2432 - acc: 0.9181 - val_loss: 0.3686 - val_acc: 0.8708
Epoch 6/10
30160/30160 [==============================] - 22s 745us/step - loss: 0.2230 - acc: 0.9245 - val_loss: 0.3637 - val_acc: 0.8751
Epoch 7/10
30160/30160 [==============================] - 22s 734us/step - loss: 0.2070 - acc: 0.9307 - val_loss: 0.3

### Evaluate the model

In [24]:
scores = model.evaluate(x_dev, y_dev, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

2755/2755 [==============================] - 1s 260us/step
Accuracy: 84.94%


### Save the model

In [25]:
outpath='models/'
os.makedirs(outpath, exist_ok=True)

In [26]:
model.save(outpath+"model.h5")

In [27]:
# serialize model to JSON
model_json = model.to_json()
with open(outpath+"model.json", "w") as json_file:
    json_file.write(model_json)

In [28]:
from tensorflow.keras import backend
from tensorflow.keras.models import load_model

In [29]:
model2=load_model("models/model.h5")

In [30]:
scores = model2.evaluate(x_dev, y_dev, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

2755/2755 [==============================] - 1s 292us/step
Accuracy: 84.94%


In [31]:
print(model.metrics_names)

['loss', 'acc']
